In [2]:
#1)VectorStoreRetriever	Vector-based	Embedding similarity search	General-purpose RAG
# ConversationalRetrievalChain

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.vectorstores import Chroma

# 1. Setup

import os
from dotenv import load_dotenv

# 2️⃣ Load API keys
load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(temperature=0)

# 2. Vector DB
with open("sample1.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

# 🧠 Split the text into smaller chunks
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(separator="\n", chunk_size=300, chunk_overlap=50)
texts = splitter.split_text(text_data)

embedding = OpenAIEmbeddings()
# vectorstore = FAISS.from_texts(texts, embedding)
vectorstore = Chroma.from_texts(texts, embedding)
retriever = vectorstore.as_retriever()

# 3. Conversational RAG chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

# 4. Memory for chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 5. Wrap RAG as a StructuredTool
def rag_tool_fn(question: str) -> str:
    return rag_chain.invoke({
        "question": question,
        "chat_history": []
    })["answer"]

rag_tool = StructuredTool.from_function(
    name="RAG_QA",
    description="Use this to answer questions about LangChain.",
    func=rag_tool_fn
)

# 6. Create agent
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# 7. Run conversation
print("1️⃣ First question")
res1 = agent.run("What is LangChain?")
print("Answer:", res1)

print("\n2️⃣ Follow-up")
res2 = agent.run("Who created it?")
print("Answer:", res2)

print("\n3️⃣ Ask again")
res3 = agent.run("Explain LangChain again simply.")
print("Answer:", res3)


1️⃣ First question


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RAG_QA
Action Input: What is LangChain?
Observation: LangChain is a framework created by Harrison Chase for building applications with LLMs. It supports RAG, agents, memory, tools, and more, and is commonly used in chatbots, document Q&A, and AI workflow.
Thought:Do I need to use a tool? No
AI: LangChain is a framework created by Harrison Chase for building applications with large language models. It supports various features like RAG, agents, memory, tools, and more, and is commonly used in chatbots, document Q&A, and AI workflow.

> Finished chain.
Answer: LangChain is a framework created by Harrison Chase for building applications with large language models. It supports various features like RAG, agents, memory, tools, and more, and is commonly used in chatbots, document Q&A, and AI workflow.

2️⃣ Follow-up


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool?